In [18]:
import pandas as pd
import sqlalchemy
from itertools import pairwise
from routes import routes
from utils import graph_utils, station_manager, visualization_utils
import networkx as nx

In [19]:
# import stations
stations = routes.extracts('../data/processed/stations.gpkg')
remover = routes.select_criteria(stations) # filter stations (same criteria as routes)
stations = station_manager.remove_stations(stations, remover) # remover as estações deletadas; aqui tem que ser o mesmo critério de routes
# make graph
G = graph_utils.generate_graph(stations)

In [20]:
engine = sqlalchemy.create_engine('sqlite:///../data/processed/database.db')

def import_query(file_path):
    with open(file_path, 'r') as file:
        return file.read()

query = """SELECT 
    o.data,
    o.id,
    o.TU,
    o.ValorTKU,
    f.ferrovia,
    f.cliente,
    f.mercadoria,
    f.origem,
    f.destino,
    f.dist_media,
    f.intermed,
    f.len_Dijkstra,
    f.rota
FROM 
    ocorrencias o
LEFT JOIN 
    fluxos_rotas_2017 f
ON 
    o.id = f.id
WHERE 
    o.data >= '2024-01-01' and o.data <= '2024-12-01';
"""
fluxos = pd.read_sql(query, engine)

In [21]:
# import excel
trem_tipo = pd.read_excel('../data/raw/temp_trem_tipo_DR.xlsx')

In [22]:
# create a column using regex to extract 'FAF' from 'Alberto Flores (FAF, MRS)':
trem_tipo['estacao_origem'] = trem_tipo['Origem'].str.extract(r'\((\w{3})')
trem_tipo['estacao_destino'] = trem_tipo['Destino'].str.extract(r'\((\w{3})')

In [23]:
trem_tipo['Ano'].unique()

array([2022, 2021, 2023, 2024, 2025])

In [24]:
fluxos[fluxos['ferrovia']=='MRS'].sort_values(by=['origem','destino'])

,data,id,TU,ValorTKU,ferrovia,cliente,mercadoria,origem,destino,dist_media,intermed,len_Dijkstra,rota
117,2024-01-01 00:00:00.000000,31758MRS,41.0,23894.0,MRS,VALE S/A,Contêiner Vazio de 20 Pés,EEL,FAR,582.801,EEL EFR FBO FJR FJC FSE FBP FQS FAR,582.801,EEL EPI EFR FBO F49 FIE FPO FSO FZN FMC FSN FB...
315,2024-01-01 00:00:00.000000,35038MRS,3.0,1748.0,MRS,MAGNESITA REFRATARIOS SA,Contêiner Vazio de 40 Pés,EEL,FAR,582.801,EEL EFR FBO FJR FJC FSE FBP FQS FAR,582.801,EEL EPI EFR FBO F49 FIE FPO FSO FZN FMC FSN FB...
485,2024-01-01 00:00:00.000000,36134MRS,94.0,54783.0,MRS,MAERSK S/A,Contêiner Vazio de 20 Pés,EEL,FAR,582.801,EEL EFR FBO FJR FJC FSE FBP FQS FAR,582.801,EEL EPI EFR FBO F49 FIE FPO FSO FZN FMC FSN FB...
530,2024-01-01 00:00:00.000000,32165MRS,9.0,5245.0,MRS,MSC MEDITERRANEAN LOGISTICA LTDA.,Contêiner Vazio de 20 Pés,EEL,FAR,582.801,EEL EFR FBO FJR FJC FSE FBP FQS FAR,582.801,EEL EPI EFR FBO F49 FIE FPO FSO FZN FMC FSN FB...
558,2024-01-01 00:00:00.000000,37239MRS,480.0,279744.0,MRS,MAERSK S/A,Contêiner Cheio de 20 Pés,EEL,FAR,582.801,EEL EFR FBO FJR FJC FSE FBP FQS FAR,582.801,EEL EPI EFR FBO F49 FIE FPO FSO FZN FMC FSN FB...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24652,2024-11-01 00:00:00.000000,87371MRS,53795.0,29020627.0,MRS,RUMO S/A,Açúcar,ZXE,ISN,539.467,ZXE ZIQ ZBV ZQB ZKE ZEV ZPG ICB ISN,539.467,ZXE ZGK ZGT ZRI ZTW ZSL ZAX ZAR ZOI ZFQ ZTI ZX...
9631,2024-05-01 00:00:00.000000,35498MRS,69.0,11381.0,MRS,COMPANHIA SIDERURGICA NACIONAL,Prd. Siderúrgicos - Outros,ZZZ,IEF,164.944,None,164.944,ZZZ ZOP ZBL Z51 ZCP ZSX ZVH ZVN ZLV ZJY OVP OC...
5445,2024-03-01 00:00:00.000000,46583MRS,27.0,5796.0,MRS,ALIANCA NAVEGAÇÃO E LOGÍSTICA LTDA,Contêiner Vazio de 40 Pés,ZZZ,IFA,214.677,None,214.677,ZZZ ZOP ZBL Z51 ZCP ZSX ZVH ZVN ZLV ZJY OVP OC...
6717,2024-04-01 00:00:00.000000,46583MRS,27.0,5796.0,MRS,ALIANCA NAVEGAÇÃO E LOGÍSTICA LTDA,Contêiner Vazio de 40 Pés,ZZZ,IFA,214.677,None,214.677,ZZZ ZOP ZBL Z51 ZCP ZSX ZVH ZVN ZLV ZJY OVP OC...


In [25]:
trem_tipo[(trem_tipo['Ano'] == 2024) & (trem_tipo['Ferrovia'] == 'MRS')].sort_values(by=['estacao_origem','estacao_destino'])

,Ferrovia,Ano,Código,Origem,Destino,Distância (km),Locos,Vagões,TB,TU,Comprimento (m),Mercadorias,estacao_origem,estacao_destino
1655,MRS,2024,KRC01,"Arará (FAR, MRS)","Dr. Joaquim Murtinho (FDM, MRS)",492,2,26,1346,495,543,"Outras - Carga Geral Não Conteinerizada, Prd. ...",FAR,FDM
1656,MRS,2024,KRP01,"Arará (FAR, MRS)","Manoel Feio (IEF, MRS)",446,2,61,4650,3445,970,"Outras - Carga Geral Não Conteinerizada, Prd. ...",FAR,IEF
1663,MRS,2024,KSV01,"Brisamar (FBA, MRS)","Barbará (FBB, MRS)",119,2,89,3496,2113,1562,"Outras - Carga Geral Não Conteinerizada, Prd. ...",FBA,FBB
1664,MRS,2024,KSC01,"Brisamar (FBA, MRS)","Dr. Joaquim Murtinho (FDM, MRS)",460,2,15,293,8,483,"Outras - Carga Geral Não Conteinerizada, Prd. ...",FBA,FDM
1657,MRS,2024,KVS11,"Barbará (FBB, MRS)","Brisamar (FBA, MRS)",119,2,32,1630,1073,534,"Outras - Carga Geral Não Conteinerizada, Prd. ...",FBB,FBA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,MRS,2024,HDN,"Pederneiras (ZPD, RMP)","Santos (ISN, MRS)",16,3,60,7350,5520,1364,Celulose,ZPD,ISN
1733,MRS,2024,JDN (ALL),"Pederneiras (ZPD, RMP)","Santos (ISN, MRS)",16,3,83,9491,7084,1486,"Açúcar, Farelo de Soja, Grãos - Milho, Soja",ZPD,ISN
1736,MRS,2024,JDN (ALL),"Rio Preto Paulista (ZRU, RMP)","Santos (ISN, MRS)",16,3,84,10046,7610,1503,"Açúcar, Farelo de Soja, Grãos - Milho, Soja",ZRU,ISN
1735,MRS,2024,QDY01,"Replan (ZZZ, RMP)","Siderúrgica do Atlântico (HSG, MRS)",595,2,40,2622,1826,464,"Outras - Carga Geral Não Conteinerizada, Prd. ...",ZZZ,HSG


In [26]:
trem_tipo['Mercadorias'].unique()

array(['Minério de Ferro',
       'Outras - Carga Geral Não Conteinerizada, Prd. Siderúrgicos',
       'Carvão Mineral, Coque', 'Bauxita', 'Contêiner',
       'Açúcar, Farelo de Soja, Grãos - Milho, Soja', 'Prd. Siderúrgicos',
       'Minério de Ferro, Outros Graneis Minerais',
       'Contêiner Cheio de 20 Pés - Perigoso', 'Areia',
       'Cimento a Granel, Cimento, Prd. Siderúrgicos', 'Celulose',
       'Soja/Farelo Soja/Milho', 'Liquidos (Alcool, Diesel)',
       'Tanque Vazios', 'Retorno Granel Vazio', 'Retorno Celulose',
       'Container 1 Nível', 'Container 2 Níveis', 'Fertilizante',
       'Granel Granel Vazio', 'Soja, Farelo Soja, Milho',
       'Retorno Granel, Açucar',
       'Retorno Granel, Soja, Farelo Soja, Milho, Açucar',
       'Retorno Container', 'Retorno Celulose Métrica',
       'Soja, Farelo Soja, Milho, Açucar', 'Açucar', 'Celulose Métrica',
       'Container', 'Alcool', 'Vagões vazios', 'Gasolina, Óleo Diesel',
       'Soja, Milho', 'Soja, Milho, Farelo de soja'

In [27]:
trem_tipo.columns

Index(['Ferrovia', 'Ano', 'Código', 'Origem', 'Destino', 'Distância (km)',
       'Locos', 'Vagões', 'TB', 'TU', 'Comprimento (m)', 'Mercadorias',
       'estacao_origem', 'estacao_destino'],
      dtype='object')

In [31]:
trem_tipo


,Ferrovia,Ano,Código,Origem,Destino,Distância (km),Locos,Vagões,TB,TU,Comprimento (m),Mercadorias,estacao_origem,estacao_destino
0,MRS,2022,NES,"Alberto Flores (FAF, MRS)","Baia Sepetiba (FXS, MRS)",508,2,134,17101,14395,1422,Minério de Ferro,FAF,FXS
1,MRS,2022,NEG,"Alberto Flores (FAF, MRS)","Guaiba (FGI, MRS)",523,2,134,17101,14395,1422,Minério de Ferro,FAF,FGI
2,MRS,2022,KRC01,"Arará (FAR, MRS)","Dr. Joaquim Murtinho (FDM, MRS)",492,2,44,1483,429,695,"Outras - Carga Geral Não Conteinerizada, Prd. ...",FAR,FDM
3,MRS,2022,KRP01,"Arará (FAR, MRS)","Manoel Feio (IEF, MRS)",446,2,33,2040,1253,521,"Outras - Carga Geral Não Conteinerizada, Prd. ...",FAR,IEF
4,MRS,2022,QLV,"Baia Sepetiba (FXS, MRS)","Volta Redonda (FVR, MRS)",119,2,72,5624,3832,1046,"Carvão Mineral, Coque",FXS,FVR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3234,RMS,2025,K06,"Uvaranas (LUS, RMS)","Iguaçu (LIC, RMS)",136,2,105,10300,6300,2200,"Açúcar, Contêiner, Farelo de Soja, Arroz, Soja...",LUS,LIC
3235,RMS,2025,F24,"Uvaranas (LUS, RMS)","Rio Negro (LRO, RMS)",141,3,120,9800,6800,1848,"Farelo de Soja, Arroz, Soja, Trigo, Milho",LUS,LRO
3236,RMS,2025,F02,"Uvaranas (LUS, RMS)","Rio Negro (LRO, RMS)",141,2,140,10300,6300,2200,"Açúcar, Contêiner, Farelo de Soja, Arroz, Soja...",LUS,LRO
3237,RMS,2025,F04,"Uvaranas (LUS, RMS)","Rio Negro (LRO, RMS)",141,3,115,8500,5750,1791,"Farelo de Soja, Arroz, Soja, Trigo, Milho",LUS,LRO


In [30]:
# pesquisa por trem tipo

for index, row in fluxos.iterrows():
    row['ferrovia']
    row['origem']
    row['destino']
    row['mercadoria']


    display(trem_tipo[(trem_tipo['Ferrovia']==row['ferrovia']) &
            (trem_tipo['Origem']==row['origem']) &
            (trem_tipo['Destino']==row['destino']) 
            # & (trem_tipo['Mercadorias'].str.contains(row['mercadoria']))            
            ])

    break




,Ferrovia,Ano,Código,Origem,Destino,Distância (km),Locos,Vagões,TB,TU,Comprimento (m),Mercadorias,estacao_origem,estacao_destino
